In [1]:
import logging
from sys import stdout

formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler = logging.StreamHandler(stdout)
console_handler.setFormatter(formatter)

logger = logging.getLogger('opensky.spark_consumer')
logger.addHandler(console_handler)
logger.setLevel('DEBUG')


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
import pandas as pd
from pyspark.sql import types as T
from pyspark.sql import functions as F
import json
import os


# # Topics/Brokers
topic_real_time_states = "real-time-states"
topic_sparse_states = 'sparse_states'
topic_enriched_real_time_states = 'enriched-real-rime-states'
# topic_test = "topic_test"
broker = "localhost:9092"

host_name = 'cnt7-naya-cdh6'
hive_host = "localhost"
hdfs_host = "localhost"
hdfs_port = 8020

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.1 pyspark-shell'

spark = SparkSession \
    .builder \
    .appName("StructuredRealTimeState") \
    .getOrCreate()


schema = T.ArrayType(T.StructType()\
                     .add("time", T.TimestampType())\
                     .add("icao24", T.StringType())\
                     .add("callsign", T.StringType())\
                     .add("last_contact", T.TimestampType())\
                     .add("longitude", T.FloatType())\
                     .add("latitude", T.FloatType())\
                     .add("baro_altitude", T.FloatType())\
                     .add("on_ground", T.IntegerType())\
                     .add("velocity", T.FloatType())\
                     .add("geo_altitude", T.FloatType())\
                     .add("squawk", T.StringType())\
                     .add("position_source", T.IntegerType()))

In [3]:
import pyarrow as pa
import re
from datetime import datetime, timedelta
import urllib
from impala.dbapi import connect

def drop_old_partitions(impala_conn: connect, table_name:str, table_src_path:str, 
                             partition_name:str, earliest_time_to_keep:datetime.timestamp):
    """
    Drop old partitions at path <table_src_path>, drop all partitions older than <earliest_time_to_keep>
    First drop partitions through impala client (impala.dbapi), then delete folders
    """
    logger.debug(f'dropping old partitions for table {table_name}, all partitions oldert than {earliest_time_to_keep}')
    
    fs = pa.hdfs.connect(
            host=hdfs_host, 
            port=hdfs_port, 
            user='hdfs', 
            kerb_ticket=None, 
            driver='libhdfs', 
            extra_conf=None)


    partition_pattern = r'(.+\=(.+))'
    partitions_paths = fs.ls(table_src_path)
    partitions_tup = [re.findall(partition_pattern, partition_path) for partition_path in partitions_paths]
    # partitions_tup is of form : (partition path, partition date)
    # urllib.parse.unquote in order to turn special chars as %3A to regular (:) , aslo flatten the list
    partitions_tup = [(pt[0][0], urllib.parse.unquote(pt[0][1])) for pt in partitions_tup if len(pt) > 0]
    partitions_dict = {date : path for path, date in partitions_tup}

    partitions_to_delete = [p_d for p_d in partitions_dict.keys() if datetime.strptime(p_d, '%Y-%m-%d %H:%M:%S') < earliest_time_to_keep]

    # here partitions to_delete holds all partitions that should be deleted

    crsr = impala_conn.cursor()

    try:
        for part_key in partitions_to_delete:
            crsr.execute(f'alter table {table_name} drop if exists partition ({partition_name}="{part_key}");')
            fs.delete(partitions_dict[part_key], recursive=True)
            logger.debug(f'deleted : {partitions_dict[part_key]}')
    except Exception as ex:
        logger.Error(ex)
    finally:
        crsr.close()
        
# earliest_time_to_keep = datetime.now() - timedelta(minutes=450)
# table_src_path = '/user/naya/FinalProject/last_hour'
# drop_old_partitions('opensky_network', "states_last_hour", table_src_path, "date_minute", earliest_time_to_keep)


import pyarrow as pa
from datetime import datetime, timedelta
from impala.dbapi import connect
from os import path


def write_to_tables(df: DataFrame, epoch_id):
    last_hour_table_name = 'last_hour'
    target_database = 'opensky_network'
    
    root_data_path = '/user/naya/FinalProject/'
    last_hour_path = path.join(root_data_path, 'last_hour')
    last_day_path = path.join(root_data_path, 'last_day')
    last_week_path = path.join(root_data_path,'last_week')
    
    logger.info(f'write_to_hive: epoch_id: {epoch_id} len: {df.count()}')
    # write to each table - minutes, hours, days
    if df.count() != 0:
        df.persist()
        logger.debug(f'trying to write to : {last_hour_path}')
        df.withColumn('date_minute', F.date_trunc('minute', df.time))\
                    .write\
                    .mode("append")\
                    .partitionBy('date_minute')\
                    .parquet(f'hdfs://localhost:8020/{last_hour_path}')
        logger.debug(f'Trying to write to : {last_day_path}')
        df.withColumn('date_hour', F.date_trunc('hour', df.time))\
                    .write\
                    .mode("append")\
                    .partitionBy('date_hour')\
                    .parquet(f'hdfs://localhost:8020/{last_day_path}')
        df.unpersist()

        impala_conn = connect(host=host_name, database = target_database, user = 'naya', password = 'naya', auth_mechanism = 'NOSASL')

        drop_old_partitions(impala_conn, 'states_last_hour', last_hour_path, 
                                 'date_minute', datetime.now() - timedelta(hours=1))
        drop_old_partitions(impala_conn, 'states_last_day', last_day_path, 
                                 'date_hour', datetime.now() - timedelta(hours=24))

        impala_crsr = impala_conn.cursor()
        try:
            for table_name in ['states_last_hour', 'states_last_day']:
                impala_crsr.execute(f'alter table {table_name} recover partitions;')
                impala_crsr.execute(f'refresh {table_name};')
        except Exception as ex:
            logger.error(ex)
        finally:
            impala_crsr.close()
            
#         crsr.execute(f'show partitions {table_name};')
#         logger.debug([d for d, *rest in crsr.fetchall()])
    
#     impala_client.table(last_hour_table_name).drop_partition('date_minute=2019-12-28 12:13:00')
#     impala_client.table(last_hour_table_name).alter()
#     impala_client.table(last_hour_table_name).refresh()



In [4]:
import geopandas as gpd
import matplotlib
import numpy as np
import pandas as pd
from multiprocessing import  Pool
from shapely.geometry.point import Point as ShapelyPoint
from pyspark.sql.functions import udf
import time

# Add country to each row
# Load countries file
COUNTRIES_GDF = gpd.read_file('./Material/ne_10m_admin_0_countries.shp')

def get_country(lon:float, lat:float) -> str:
    if lon is None or lat is None:
        return ""
    
    #convert to aheply point
    point = ShapelyPoint(lon, lat)
    country = COUNTRIES_GDF['NAME'][COUNTRIES_GDF['geometry'].contains(point)]
    # Need to handle cases when no country found
    country = country.item() if len(country) > 0 else ""

    return country

def to_json_cols(cols_list):
    out_dict = {}
    for col in cols_list:
        F.to_json(col)

udf_get_country = udf(get_country, T.StringType())

def enrich_add_country_column(df, epoch_id):
    logger.debug(f'in enrich_add_country_column epoch_id:{epoch_id}, with len(df): {df.count()}')
    if df is None or df.count() <= 0:
        return
    minidf = df.limit(100)
    start_enrich = time.time()
#     df = df.withColumn('country', udf_get_country('longitude', 'latitude'))
    df = minidf.withColumn('country', udf_get_country('longitude', 'latitude'))
    end_enrich = time.time()
    logger.debug(f'enriched {df.count()} records, within {end_enrich - start_enrich} seconds')
    test_df = df.toJSON().toDF('value')
    logger.debug(f'this is the df: {test_df.show()}')
#     rdd = df.rdd.map(lambda row: row.asDict())
#     logger.debug(f'rdd is {rdd.collect()}')
    # send to kafka
#     df.select(F.toJSON())\
#         .write\
#         .format("kafka") \
#         .option("kafka.bootstrap.servers", broker) \
#         .option("topic", topic_enriched_real_time_states) \
#         .save()


In [6]:
# schema = T.ArrayType(T.StructType()\
#                      .add("time", T.TimestampType())\
#                      .add("icao24", T.StringType())\
#                      .add("callsign", T.StringType())\
#                      .add("last_contact", T.TimestampType())\
#                      .add("longitude", T.FloatType())\
#                      .add("latitude", T.FloatType())\
#                      .add("baro_altitude", T.FloatType())\
#                      .add("on_ground", T.IntegerType())\
#                      .add("velocity", T.FloatType())\
#                      .add("geo_altitude", T.FloatType())\
#                      .add("squawk", T.StringType())\
#                      .add("position_source", T.IntegerType()))

# from pyspark.sql import functions as F

# df = spark \
#     .readStream \
#     .format("kafka") \
#     .option("kafka.bootstrap.servers", broker) \
#     .option("subscribe", topic_real_time_states) \
#     .option("startingOffsets", "latest")\
#     .load()

# state_vectors_df = df.select(F.from_json(F.col("value").cast("string"), schema).alias("value"))\
#                         .select((F.explode("value").alias("value")))\
#                         .select("value.*")

enrich_with_countries = state_vectors_df\
            .writeStream\
            .foreachBatch(enrich_add_country_column)\
            .start()


2020-01-14 22:52:51,897 - opensky.spark_consumer - DEBUG - in enrich_add_country_column epoch_id:0, with len(df): 0
2020-01-14 22:53:10,933 - opensky.spark_consumer - DEBUG - in enrich_add_country_column epoch_id:1, with len(df): 7906
2020-01-14 22:53:11,836 - opensky.spark_consumer - DEBUG - enriched 100 records, within 0.4064912796020508 seconds
2020-01-14 22:53:17,931 - opensky.spark_consumer - DEBUG - rdd is [{'time': datetime.datetime(2020, 1, 14, 22, 52, 55), 'icao24': 'e48d22', 'callsign': 'PRMJM', 'last_contact': datetime.datetime(2020, 1, 14, 22, 52, 19), 'longitude': -43.82320022583008, 'latitude': -18.910600662231445, 'baro_altitude': 13037.8203125, 'on_ground': 0, 'velocity': 214.16000366210938, 'geo_altitude': 13868.400390625, 'squawk': None, 'position_source': 0, 'country': 'Brazil'}, {'time': datetime.datetime(2020, 1, 14, 22, 52, 55), 'icao24': 'a3b87f', 'callsign': 'DAL797', 'last_contact': datetime.datetime(2020, 1, 14, 22, 52, 55), 'longitude': -88.35340118408203, 'l

In [6]:
topic_enriched_real_time_states = 'enriched-real-rime-states'

2020-01-14 19:34:18,297 - opensky.spark_consumer - DEBUG - in enrich_add_country_column epoch_id:0, with len(df): 0
2020-01-14 19:35:52,824 - opensky.spark_consumer - DEBUG - in enrich_add_country_column epoch_id:1, with len(df): 8002
2020-01-14 19:35:53,916 - opensky.spark_consumer - DEBUG - enriched 8002 records, within 0.5948793888092041 seconds


In [28]:
print(enrich_with_countries.isActive)
if enrich_with_countries.exception() != None:
    print(enrich_with_countries.exception().desc)

# enrich_with_countries.stop()


False


In [7]:
enrich_with_countries.stop()
spark.streams.active 
# for s in spark.streams.active:
#     s.stop()

[]

In [12]:
from pyspark.sql import functions as F

df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", broker) \
    .option("subscribe", topic_real_time_states) \
    .option("startingOffsets", "latest")\
    .load()

state_vectors_df = df.select(F.from_json(F.col("value").cast("string"), schema).alias("value"))\
                        .select((F.explode("value").alias("value")))\
                        .select("value.*")

# agg_count = state_vectors_df.withWatermark("time", "1 minute").groupBy("time").count()
# query = agg_count\
#             .writeStream\
#             .outputMode("append")\
#             .format("console") \
#             .start()

# # Write to parquet file
# TBD - handle target path creation
parquet_path = 'hdfs://cnt7-naya-cdh6.org:8020/FinalProject/parquet_archive'
parquet_checkpoint_path = "/home/naya/parquet_checkpoint"
parquet_write = state_vectors_df\
                .withColumn('date_hour', F.date_trunc('hour', state_vectors_df.time))\
                .writeStream\
                .outputMode("append")\
                .format("parquet")\
                .partitionBy('date_hour')\
                .option("checkpointLocation", parquet_checkpoint_path)\
                .option("path", parquet_path)\
                .start()

tables_write = state_vectors_df\
                .writeStream\
                .foreachBatch(write_to_tables)\
                .start()


enrich_with_countries = state_vectors_df\
            .writeStream\
            .foreachBatch(enrich_add_country_column)\
            .start()
# query.awaitTermination()


In [20]:
# query.isActive
# parquet_write.recentProgress
# query.lastProgress
# parquet_write.exception()


False


pyspark.sql.utils.StreamingQueryException('An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):\n  File "/home/naya/anaconda3/lib/python3.6/site-packages/py4j/java_gateway.py", line 2381, in _call_proxy\n    return_value = getattr(self.pool[obj_id], method)(*params)\n  File "/home/naya/anaconda3/lib/python3.6/site-packages/pyspark/sql/utils.py", line 191, in call\n    raise e\n  File "/home/naya/anaconda3/lib/python3.6/site-packages/pyspark/sql/utils.py", line 188, in call\n    self.func(DataFrame(jdf, self.sql_ctx), batch_id)\n  File "<ipython-input-18-88cb41939030>", line 40, in enrich_add_country_column\n    aircraft_gdf = gpd.GeoDataFrame(df[[\'icao24\', \'callsign\', \'time\', \'on_ground\']], geometry = gpd.points_from_xy(df[\'longitude\'], df[\'latitude\']))\n  File "/home/naya/anaconda3/lib/python3.6/site-packages/geopandas/array.py", line 185, in _points_from_xy\n    if not len(x) == len(y):\nTypeError: object of type \'Column\' has n

2020-01-13 21:03:14,539 - opensky.spark_consumer - DEBUG - Trying to write to : /user/naya/FinalProject/last_day
2020-01-13 21:03:15,067 - opensky.spark_consumer - DEBUG - dropping old partitions for table states_last_hour, all partitions oldert than 2020-01-13 20:03:15.067684
2020-01-13 21:03:15,188 - opensky.spark_consumer - DEBUG - dropping old partitions for table states_last_day, all partitions oldert than 2020-01-12 21:03:15.188579
2020-01-13 21:03:27,614 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 4323 len: 9717
2020-01-13 21:03:28,138 - opensky.spark_consumer - DEBUG - trying to write to : /user/naya/FinalProject/last_hour
2020-01-13 21:03:28,721 - opensky.spark_consumer - DEBUG - Trying to write to : /user/naya/FinalProject/last_day
2020-01-13 21:03:28,849 - opensky.spark_consumer - DEBUG - dropping old partitions for table states_last_hour, all partitions oldert than 2020-01-13 20:03:28.849427
2020-01-13 21:03:28,969 - opensky.spark_consumer - DEBUG - dropping 

2020-01-13 21:07:12,825 - opensky.spark_consumer - DEBUG - Trying to write to : /user/naya/FinalProject/last_day
2020-01-13 21:07:12,946 - opensky.spark_consumer - DEBUG - dropping old partitions for table states_last_hour, all partitions oldert than 2020-01-13 20:07:12.946014
2020-01-13 21:07:12,988 - opensky.spark_consumer - DEBUG - deleted : /user/naya/FinalProject/last_hour/date_minute=2020-01-13 20%3A07%3A00
2020-01-13 21:07:13,091 - opensky.spark_consumer - DEBUG - dropping old partitions for table states_last_day, all partitions oldert than 2020-01-12 21:07:13.091564
2020-01-13 21:07:29,933 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 4335 len: 9824
2020-01-13 21:07:30,464 - opensky.spark_consumer - DEBUG - trying to write to : /user/naya/FinalProject/last_hour
2020-01-13 21:07:31,040 - opensky.spark_consumer - DEBUG - Trying to write to : /user/naya/FinalProject/last_day
2020-01-13 21:07:31,551 - opensky.spark_consumer - DEBUG - dropping old partitions for table s

2020-01-13 21:11:34,599 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 4346 len: 9093
2020-01-13 21:11:35,113 - opensky.spark_consumer - DEBUG - trying to write to : /user/naya/FinalProject/last_hour
2020-01-13 21:11:36,091 - opensky.spark_consumer - DEBUG - Trying to write to : /user/naya/FinalProject/last_day
2020-01-13 21:11:36,212 - opensky.spark_consumer - DEBUG - dropping old partitions for table states_last_hour, all partitions oldert than 2020-01-13 20:11:36.212811
2020-01-13 21:11:36,332 - opensky.spark_consumer - DEBUG - dropping old partitions for table states_last_day, all partitions oldert than 2020-01-12 21:11:36.332128
2020-01-13 21:11:55,171 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 4347 len: 9072
2020-01-13 21:11:55,699 - opensky.spark_consumer - DEBUG - trying to write to : /user/naya/FinalProject/last_hour
2020-01-13 21:11:56,276 - opensky.spark_consumer - DEBUG - Trying to write to : /user/naya/FinalProject/last_day
2020-01-13 21:11:56,4

In [17]:
print(parquet_write.isActive)
# print(query.isActive)

True
2020-01-13 20:59:47,178 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 4313 len: 9481
2020-01-13 20:59:47,716 - opensky.spark_consumer - DEBUG - trying to write to : /user/naya/FinalProject/last_hour
2020-01-13 20:59:48,270 - opensky.spark_consumer - DEBUG - Trying to write to : /user/naya/FinalProject/last_day
2020-01-13 20:59:48,383 - opensky.spark_consumer - DEBUG - dropping old partitions for table states_last_hour, all partitions oldert than 2020-01-13 19:59:48.383872
2020-01-13 20:59:48,503 - opensky.spark_consumer - DEBUG - dropping old partitions for table states_last_day, all partitions oldert than 2020-01-12 20:59:48.502901
2020-01-13 21:00:10,718 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 4314 len: 9531
2020-01-13 21:00:11,238 - opensky.spark_consumer - DEBUG - trying to write to : /user/naya/FinalProject/last_hour
2020-01-13 21:00:11,808 - opensky.spark_consumer - DEBUG - Trying to write to : /user/naya/FinalProject/last_day
2020-01-13 21:00

In [ ]:
query.stop()

In [100]:
parquet_write.stop()
tables_write.stop()

In [9]:
tables_write.isActive

True

2020-01-12 21:30:24,015 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2357 len: 9507
2020-01-12 21:30:44,032 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2358 len: 9516
2020-01-12 21:31:09,030 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2359 len: 9490
2020-01-12 21:31:40,457 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2360 len: 9511
2020-01-12 21:31:54,466 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2361 len: 9426
2020-01-12 21:32:09,841 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2362 len: 9452
2020-01-12 21:32:25,817 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2363 len: 9452
2020-01-12 21:32:43,204 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2364 len: 9455
2020-01-12 21:33:01,248 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2365 len: 9450
2020-01-12 21:33:19,070 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2366 len: 9451
2020-01-12 21:33:41,

2020-01-12 22:04:26,670 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2441 len: 8242
2020-01-12 22:05:08,289 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2442 len: 8227
2020-01-12 22:05:46,170 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2443 len: 8197
2020-01-12 22:06:26,687 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2444 len: 8187
2020-01-12 22:06:40,436 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2445 len: 8185
2020-01-12 22:06:57,046 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2446 len: 8209
2020-01-12 22:07:19,734 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2447 len: 8219
2020-01-12 22:07:43,804 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2448 len: 8254
2020-01-12 22:08:16,560 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2449 len: 8269
2020-01-12 22:08:46,048 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2450 len: 8252
2020-01-12 22:09:11,

In [5]:
hive_write.exception()

In [22]:

hive_write.stop()

select date_trunc('hour', time), cast(last_contact as timestamp) from states limit 20;


CREATE EXTERNAL TABLE playground.states_hourly
(time TIMESTAMP, icao24 STRING, callsign STRING, last_contact TIMESTAMP,
longitude FLOAT, latitude FLOAT, baro_altitude FLOAT, on_ground INT,   
velocity FLOAT, geo_altitude FLOAT, squawk STRING, position_source INT)
PARTITIONED BY (date_hour string)
STORED AS PARQUET 
LOCATION 'hdfs://cnt7-naya-cdh6.org:8020/FinalProject/hourly';


SHOW CREATE TABLE states;